# fMRI to fNIRS

In this notebook, we develop a pipeline for turning fMRI data into pseudo-fNIRS data, and we apply it to the NSD.

In [22]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

In [28]:
# specify subject data
subjectID  = '01'
sessionID  = '21'
runID      = '01'

# create directories to store data
anat_path = 'data/sub' + subjectID + '/anat/'
run_path  = 'data/sub' + subjectID + '/func/fmri/sess' + sessionID + '/run' + runID + '/'
if not os.path.isdir(anat_path): os.makedirs(anat_path)
if not os.path.isdir(run_path):  os.makedirs(run_path)

## 1. Create segmented mask

### Install FSL library

In [24]:
## uncomment if library not installed
# !sudo apt -qq install file
# !wget https://fsl.fmrib.ox.ac.uk/fsldownloads/fslinstaller.py
# !python3 fslinstaller.py

from pathlib import Path
fslpath = Path(os.getcwd()).parents[1].as_posix()+'/fsl'
os.environ["FSLDIR"] = fslpath
os.environ["FSLOUTPUTTYPE"] = "NIFTI_GZ"
os.environ["PATH"] += os.pathsep + os.path.join(fslpath, 'bin')
!. ${FSLDIR}/etc/fslconf/fsl.sh

### Retrieve subject data

In [ ]:
resolution = '1pt0' # 1.0 (for retrieving data from NSD)

# specify urls and filenames
anat_url = f"https://natural-scenes-dataset.s3.amazonaws.com/nsddata/ppdata/subj{subjectID}/anat/"
fmri_url = f"https://natural-scenes-dataset.s3.amazonaws.com/nsddata_rawdata/sub-{subjectID}/ses-nsd{sessionID}/func/"
head_mask_filename = f"brainmask_{resolution}.nii.gz"
t1_filename        = f"T1_{resolution}_masked.nii.gz"
fmri_filename      = f"sub-{subjectID}_ses-nsd{sessionID}_task-nsdcore_run-{runID}_bold.nii.gz"
resting_filename = f"sub-{subjectID}_ses-nsd{sessionID}_task-rest_run-{1}_bold.nii.gz"

# pull structural data from NSD
if not os.path.isfile(anat_path+head_mask_filename): 
    !wget -P $anat_path $anat_url$head_mask_filename 
if not os.path.isfile(anat_path+t1_filename): 
    !wget -P $anat_path $anat_url$t1_filename

# generate brain masks (~5s)
t1_brain_filename = f"T1_{resolution}_masked_brain"
if not os.path.isfile(anat_path+t1_brain_filename+'.nii.gz'):
    !bet $anat_path$t1_filename $anat_path$t1_brain_filename -m

# segment brain tissue into csf, grey and white matter (~5m8s)
if not os.path.isfile(anat_path+t1_brain_filename+'_seg.nii.gz'):
    !fast $anat_path$t1_brain_filename

# pull functional data from NSD
if not os.path.isfile(run_path+fmri_filename): 
    !wget -P $run_path $fmri_url$fmri_filename
if not os.path.isfile(run_path+resting_filename): 
    !wget -P $run_path $fmri_url$resting_filename

### Create mask in `anat` space

In [ ]:
# convert masks to numpy arrays
head_mask_nib = nib.load(anat_path+head_mask_filename)
brain_seg_nib = nib.load(anat_path+t1_brain_filename+"_seg.nii.gz")

# create segmented model
t1_head_seg = np.zeros(head_mask_nib.get_fdata().shape)
t1_head_seg[head_mask_nib.get_fdata()==1] = 1
t1_head_seg[brain_seg_nib.get_fdata()==1] = 2
t1_head_seg[brain_seg_nib.get_fdata()==2] = 3
t1_head_seg[brain_seg_nib.get_fdata()==3] = 4
t1_head_seg = nib.Nifti1Image(t1_head_seg, brain_seg_nib.affine, brain_seg_nib.header)

# save anat mask
t1_head_seg_filename = f'T1_{resolution}_masked_'+'head_seg.nii.gz'
if not os.path.isfile(anat_path+t1_head_seg_filename): nib.save(t1_head_seg, anat_path+t1_head_seg_filename)

# plot anat mask
sidx = t1_head_seg.shape[2]//2
print(t1_head_seg.shape)
plt.imshow(t1_head_seg.get_fdata()[:,:,sidx]); plt.colorbar()
plt.title('Head mask in anat space'); plt.show()

### Create mask in `func` space

In [30]:
# register segmented head model with fmri data
fmri_head_seg_filename = f'sub-{subjectID}_ses-nsd{sessionID}_task-nsdcore_run-{runID}_bold_head_seg.nii.gz'
if not os.path.isfile(run_path+fmri_head_seg_filename):
    !flirt -in $anat_path$t1_head_seg_filename -ref $run_path$fmri_filename -out $run_path$fmri_head_seg_filename

# save func mask
seg2fmri = nib.load(run_path+fmri_head_seg_filename)

# plot func mask


# seg2fmri = np.round(nib.load("seg2fmri.nii.gz").get_fdata()).astype('uint8')
# fidx = seg2fmri.shape[2]//2
# print(seg2fmri.shape)
# plt.imshow(seg2fmri[:,:,fidx]); plt.colorbar()
# plt.title('Head mask in functional space'); plt.show()

## 2. Forward solve

INPUT: 3D head mask is found in `data/subID/func/fmri/sessID/runID` 

OUTPUT: Measurement data in `data/subID/func/fnirs/sessID/runID`

In [ ]:
# install libraries
!pip install numpy pmcx jdata bjdata matplotlib

In [ ]:
import pmcx
pmcx.__version__   # print imported pmcx version number

In [ ]:
pmcx.gpuinfo() # see gpu info

In [27]:
def get_optode_locations(brain_seg: np.ndarray):
    """
    Get optode locations from brain segmentation

    Parameters
    ----------
    brain_seg : numpy array (3D)

    Returns
    -------
    srcpositions : list of source positions
    srcdirections : list of source directions
    detpositions : list of detector positions
    """
    srcpositions = [[30,30,0]] # source position
    srcdirections = [[0,0,1]] # source direction
    detpositions = [[25,25,0]] # detector position

    return srcpositions, srcdirections, detpositions

In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

In [ ]:
# todo: integrate steve's source detectors

# generate a random volume of size 2 x Nx x Ny x Nz float32
# each element should be a random number between 0.5 and 1
vol = np.random.rand(2, 100, 100, 100).astype('float32') + 0.5

srcpositions, srcdirections, detpositions = get_optode_locations(vol)

n = 1.37 # index of refraction
g = 0.9 # anisotropy factor

cfg = {
       'nphoton': 1000000,
       'vol': vol,
       'tstart':0,
       'tend': 5e-9,
       'tstep': 5e-9,
       'srcpos': srcpositions[0],
       'srcdir': srcdirections[0],
       'prop': [[0,0,1,1],[0,0,g,n]]
       }